In [1]:
!pip install -r requirements.txt

     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/987.4 kB ? eta -:--:--
     --------- ---------------------------- 235.5/987.4 kB 7.3 MB/s eta 0:00:01
     ---------------------- --------------- 573.4/987.4 kB 7.2 MB/s eta 0:00:01
     -------------------------------------  972.8/987.4 kB 7.7 MB/s eta 0:00:01
     -------------------------------------- 987.4/987.4 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/98.2 kB ? eta -:--:--
     ---------------------------------------- 98.2/98.2 kB 5.9 MB/s eta 0:00:00
     ---------------------------------------- 0.0/96.9 kB ? eta -:--:--
     ---------------------------------------- 96.9/96.9 kB 5.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from pytube import YouTube
from youtube_transcript_api import YouTubeTranscriptApi
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import scrapetube

folderID = "16k9dQQo_izGEjcpjlStbbgJZOvh4jRIB"

In [3]:
# API to download YouTube videos
def downloadVideo(url):
    yt = YouTube(url)
    file1Name = yt.video_id + ".mp4"
    file2Name = yt.video_id + ".txt"

    # Filtering for videos with video and audio
    stream = yt.streams.filter(progressive=True).get_by_itag(22)
    stream.download(filename=file1Name)

    # English transcription
    srt = YouTubeTranscriptApi.get_transcript(yt.video_id, languages=['en'])

    # Write transcription to readable format
    with open(file2Name, "w") as outFile:
        for caption in srt:
            start = round(caption["start"], 2)
            end = round(start + caption["duration"], 2)
            text = caption["text"]
            line = str(start) + " -> " + str(end) + "\n" + text + "\n\n"
            outFile.write(line)

    gauth = GoogleAuth()
    drive = GoogleDrive(gauth)

    # Upload files to Google Drive
    file1Metadata = {"name":file1Name, "mimeType":"video/mp4", "parents":[{"id":folderID}]}
    file2Metadata = {"name":file2Name, "mimeType":"file/txt", "parents":[{"id":folderID}]}
    
    file1 = drive.CreateFile(metadata=file1Metadata)
    file1.SetContentFile(file1Name)
    file1.Upload()
    
    file2 = drive.CreateFile(metadata=file2Metadata)
    file2.SetContentFile(file2Name)
    file2.Upload()


In [ ]:
# Retrieve latest 50 uploads by the NPR youtube channel
videos = scrapetube.get_channel('UCJnS2EsPfv46u1JR8cnD0NA')
urls = []
MAX_VIDEOS = 50
videoCount = 0

for video in videos:
    urls.append("https://www.youtube.com/watch?v=" + video["videoId"] + "&ab_channel=NPR")
    videoCount += 1
    if videoCount >= MAX_VIDEOS:
        break

for url in urls:
    downloadVideo(url)

In [ ]:
import numpy as np
import cv2
import torch
import os
import keras
from torchvision import transforms
from keras import layers
from sklearn.model_selection import test_train_split